# 基于服务器IPS参数的强化学习(RL)推荐系统 - Version 20170823

## 1. 实验背景介绍


## 2. 实验设计


## 3. 测试结果介绍 （400组=10种workloads\*8种压力\*5循环）

在 IPS-rating-final.csv 文件中，

1. 第一列代表各种 workloads ，如 workload_id=0 代表 名称为 bwaves 的workload.
2. 第二列代表各种测试压力, 如 pressure_id=0 代表 CPU 压力。
3. 第三列代表 （某压力下IPS／单独运行时IPS） 的百分制评分

注：当前测试的七种压力（i.e., CPU, LL3 cache, memory）均由 ibench 测试集提供。

## 4. 目标产出： 

### 一个 学习了400次之后的Q-table，当第401次任务来的时候，根据Q-table选择“去那里执行，最好最快”。


## (1) 为 jupyter notebook 载入 python 需要的库

In [1]:
import numpy as np # linear algebra
np.random.seed(0)
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# import matplotlib.pyplot as plt
# import seaborn as sns
# from sklearn.metrics import mean_squared_error
# import math
# from scipy import stats
# %matplotlib inline  

## (2) 载入测试数据


In [2]:
# Load data from disk

names = ['workload_id', 'pressure_id', 'rating']
df = pd.read_csv('/Users/dong/Desktop/PhD-Research/体系-数据分析/Reinforcement_Learning_IPS_rating.csv',delimiter=",", names=names)

print(df.shape)

num_workloads = df.workload_id.unique().shape[0]
num_pressures = df.pressure_id.unique().shape[0]

print(num_workloads, "kinds of workloads")
print(num_pressures, "kinds of pressures")

(400, 3)
10 kinds of workloads
8 kinds of pressures


## (3) 观察数据

In [3]:
df.head()

,workload_id,pressure_id,rating
0,0,0,54
1,1,0,50
2,4,0,50
3,6,0,50
4,7,0,95


In [4]:
df.tail()

,workload_id,pressure_id,rating
395,5,7,100
396,8,7,100
397,9,7,100
398,3,7,100
399,2,7,100


## (4) 建立 Q-table

### 在Q-table中，纵轴是各类states，横轴是各类action，RL这种算法的产出就是在某时刻，当遇到某种state，我们应该根据Q-table作出怎样的决策。

### 本次设计，“我们认为” 第一列workload_id为state，pressure_id为我们愿意选择的action（愿意在那种pressure下）。

In [1]:
class QLearningTable:
    def __init__(self, actions, learning_rate=0.01, reward_decay=0.9, e_greedy=0.9):
        # In Hipster, e_greedy=1.0, so that each time, the best one(action) will always be choosed.
        self.actions = actions  # a list
        self.lr = learning_rate  # 学习率
        self.gamma = reward_decay # 奖励衰减
        self.epsilon = e_greedy  # 贪婪度
        self.q_table = pd.DataFrame(columns=self.actions)  # 初始 q_table

    def choose_action(self, observation):
        # action selection
        # 选择 action      
        self.check_state_exist(observation)
        # 检测本 state 是否在 q_table 中存在(见后面标题内容)
        if np.random.uniform() < self.epsilon:
            # choose best action
            state_action = self.q_table.ix[observation, :]
            state_action = state_action.reindex(np.random.permutation(state_action.index))     
            # some actions have same value
            action = state_action.argmax()
        else:
            # choose random action
            action = np.random.choice(self.actions)
        return action

    def learn(self, s, a, r, s_):
        # s -- current state
        # a -- action 
        # r -- reward
        # s_ -- state at next 
        self.check_state_exist(s_)
        q_predict = self.q_table.ix[s, a]
        # if s_ != 'terminal':
        q_target = r + self.gamma * self.q_table.ix[s_, :].max()  # next state is not terminal
        #else:
            #q_target = r  # next state is terminal
        self.q_table.ix[s, a] += self.lr * (q_target - q_predict)  # update

    def check_state_exist(self, state):
        # 检测本 state 是否在 q_table 中存在(见后面标题内容)
        # 在 本推荐系统中，是检测 一个 workload 是否存在，不存在
        if state not in self.q_table.index:
            # append new state to q table
            self.q_table = self.q_table.append(
                pd.Series(
                    [0]*len(self.actions),
                    index=self.q_table.columns,
                    name=state,
                )
            )

## 在时序的400个回合中，不断更新 Q-table

In [10]:
def update():
    # 学习 100 回合
    for episode in range(100):
        # 初始化 state 的观测值
        observation = env.reset()

        while True:
            # 更新可视化环境
            # env.render()

            # RL 大脑根据 state 的观测值挑选 action
            action = RL.choose_action(str(observation))

            # 探索者在环境中实施这个 action, 并得到环境返回的下一个 state 观测值, reward 和 done (是否是掉下地狱或者升上天堂)
            observation_, reward, done = env.step(action)

            # RL 从这个序列 (state, action, reward, state_) 中学习
            RL.learn(str(observation), action, reward, str(observation_))

            # 将下一个 state 的值传到下一次循环
            observation = observation_

            # 如果掉下地狱或者升上天堂, 这回合就结束了
            if done:
                break

    # 结束游戏并关闭窗口
    # print('game over')
    # env.destroy()



## 此时，401次来临，该如何“做出推荐”

In [11]:
if __name__ == "__main__":
    # 定义环境 env 和 RL 方式
    env = Maze()
    RL = QLearningTable(actions=list(range(env.n_actions)))

    # 开始可视化环境 env
    env.after(100, update)
    env.mainloop()

NameError: name 'Maze' is not defined